# Intensity Calculation

In [3]:
import utils 
import torch
import os
import pickle
import csv

# Data

In [4]:
data_path = f'data/conttime_1000_Seq_5_Dim_50_MaxLen_8_LSTMUnits.pkl'
dim_process = 5

In [17]:
interarrival_seqs, event_types_seqs, length_seqs = utils.open_pkl_file(data_path, 'test')

In [9]:
# Padding
interarrival_seqs, event_types_seqs = utils.padding_full(
        interarrival_seqs, 
        event_types_seqs, 
        length_seqs, 
        dim_process
    )

In [12]:
interarrival_batch = interarrival_seqs[0:5]
event_types_batch = event_types_seqs[0:5]
length_seqs_batch = length_seqs[0:5]

# Generating Simulated Data

In [13]:
sim_diffs_seqs, sim_diffs_index, total_duration_seqs = utils.generate_simulation(interarrival_batch, length_seqs_batch)

# Forward

In [16]:
model = torch.load('./models/conttime_100_Seq_2_Dim_100_MaxLen_8_LSTMUnits.pt')


Conttime(
  (emb): Embedding(3, 8)
  (lstm_cell): CTLSTMCell(
    (linear): Linear(in_features=16, out_features=56, bias=True)
  )
  (hidden_lambda): Linear(in_features=8, out_features=2, bias=True)
)

# Calculating Estimated Intensity Neural 

In [ ]:
hidden_states, *_ = model(event_types_seqs, interarrival_seqs)

In [ ]:
lambda_all = F.softplus(model.hidden_lambda(hidden_states))

# Calculating Intensity Hawkes (Actual)

In [ ]:
def compute_intensity_hawkes(self, time_current):
        # Initialize intensity with background rate (already is not SFT)
        self.intensity = np.copy(self.mu)
        
        # Looping over previous events
        for event in self.one_sequence:
            # Calculate intensity form equation (2)
            time_since_start = event['time_since_start']
            event_type = event['event_type']
            change_time = time_current - time_since_start
            
            decay_frac = np.exp(-self.delta[:, event_type] * change_time)

            self.intensity += np.copy(self.alpha[:, event_type] * decay_frac)
            
            if model = "hawkesinhib":
                self.intensity = self.soft_relu(self.intensity)
        
        return self.intensity

In [ ]:
intensity_list = []

for time in sequence:
    intensity = compute_intensity_hawkes(time)
    intensity_list.append(intensity)

# Calculating Intensity Neural (Actual)

In [ ]:
with open("./gen_models/conttime_100_Seq_2_Dim_100_MaxLen_8_LSTMUnits.pkl", "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)

In [ ]:
data

In [ ]:
generator_settings = {
        'dim_process': dim_process,
        'lstm_units': lstm_units,
        'seed': seed,
        'pretrain_model_path': pretrain_model_path,
        'sum_for_time': sum_for_time,
        'args': None  # Update this if 'args' is something specific
    }



In [ ]:
gen_model = seq_generator.NeuralHawkesCTLSTM(generator_settings)

for time in sequence:
    

# Data for R

In [7]:
interarrival_seqs[0:2]

[tensor([0.2480, 0.6965, 0.0386, 0.2882, 0.1562, 0.3219, 0.2465, 0.3069, 0.0895,
         0.2165, 0.3193, 0.1113, 0.0127, 0.3950, 0.1227, 0.3190, 0.2185, 0.0250,
         0.0506, 0.0367, 0.0552, 0.0735, 0.0135, 0.0265, 0.0041, 0.0851, 0.2348,
         0.1769, 0.0548, 0.0146, 0.2199, 0.1452, 0.2108, 0.0197, 0.2057, 0.0272,
         0.0598, 0.0933, 0.1361, 0.5554, 0.2824, 0.0402, 0.0202, 0.3998, 0.0644,
         0.3322, 0.0838, 0.0805]),
 tensor([0.2725, 0.0124, 0.0933, 0.0879, 0.1025, 0.0491, 0.0934, 0.2177, 0.3190,
         0.0821, 0.1706, 0.1492, 0.3024, 1.1722, 0.0316, 0.0770, 0.0460, 0.4269,
         0.6420, 0.0486, 0.3461, 0.2229, 0.1622])]

In [19]:
# Specify the file path
file_path = 'length_seqs_train.csv'

# Write data to CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(length_seqs)

In [29]:
interarrival_seqs, event_types_seqs = utils.padding_full(
        interarrival_seqs, 
        event_types_seqs, 
        length_seqs, 
        dim_process
    )

In [26]:
torch.save(interarrival_seqs, 'interarrival_seqs.pt')

In [27]:
torch.load('interarrival_seqs.pt')

tensor([[0.0000, 0.2480, 0.6965,  ..., 0.0805, 0.0000, 0.0000],
        [0.0000, 0.2725, 0.0124,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1545, 0.1190,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.1723, 0.2842,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0263, 0.3160,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2332, 0.0590,  ..., 0.0000, 0.0000, 0.0000]])

In [ ]:
# Load the model
model = torch.load("conttime_100_Seq_2_Dim_100_MaxLen_8_LSTMUnits.pt")

# Print the model
print(model)
